In [3]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import dnn_test
from dnn_test import sigmoid,sigmoid_backward,relu,relu_backward
np.random.seed(1)

In [4]:
def init_params(n_x,n_h,n_y): ####for single hidden layer 
    np.random.seed(2)
    w1=np.random.randn(n_h,n_x)*0.01
    b1=np.zeros((n_h,1))
    w2=np.random.randn(n_y,n_h)*0.01
    b2=np.zeros((n_y,1))
    
    assert(w1.shape==(n_h,n_x))
    assert(w2.shape==(n_y,n_h))
    assert(b1.shape==(n_h,1))
    assert(b2.shape==(n_y,1))
    
    params={
        "w1":w1,
        "w2":w2,
        "b1":b1,
        "b2":b2
    }
    return params

In [5]:
def init_params_deep(n_layers): #n_layers= no of layes in deep network
    np.random.seed(3)
    params={}
    l=len(n_layers)
    
    for i in range(1,l):
        params["w"+str(i)]=np.random.randn(n_layers[i],n_layers[i-1])*0.01
        params["b"+str(i)]=np.zeros((n_layers[i],1))
        
        assert(params["w"+str(i)].shape==(n_layers[i],n_layers[i-1]))
        assert(params["b"+str(i)].shape==(n_layers[i],1))
        
    return params
        
    

In [6]:
def lin_forword(a,w,b):
    z=np.dot(w,a)+b
    assert(z.shape==(w.shape[0],a.shape[1]))
    save=(a,w,b)
    return z,save

In [8]:
def lin_acti_forword(a_prev,w,b,activation):
    if activation=="sigmoid":
        z,l_save=lin_forword(a_prev,w,b)
        a,a_save=sigmoid(z)
    elif activation=="relu":
        z,l_save=lin_forword(a_prev,w,b)
        a,a_save=relu(z)
        
    assert(a.shape==(w.shape[0],a_prev.shape[1]))
    save=(l_save,a_save)
    return a,save

In [9]:
def l_forword_model(x,params):
    save_d=[]
    a=x
    l=len(params)//2
    
    for i in range(1,l):
        a_prev=a
        a,save=lin_acti_forword(a_prev,params['w'+str(i)],params["b"+str(i)],"relu")
        save_d.append(save)
    
    al,save=lin_acti_forword(a,params["w"+str(l)],params["b"+str(l)],"sigmoid")
    
    save_d.append(save)
    
    assert(al.shape==(1,x.shape[1]))
    return al,save_d

In [10]:
def com_cost(al,y):
    m=y.shape[1]
    cost=-(np.dot(y,np.log(al).T)+np.dot((1-y),np.log(1-al).T))/m
    cost=np.squeeze(cost)
    assert(cost.shape==())
    return cost

def lin_backward(dz,save):
    a_prev,w,b=save
    m=a_prev.shape[1]
    
    dw=np.dot(dz,a_prev.T)/m
    db=np.sum(dz,axis=1,keepdims=True)/m
    da_prev=np.dot(w.T,dz)
    
    assert(da_prev.shape==a_prev.shape)
    assert(dw.shape==w.shape)
    assert(db.shape==b.shape)
    return da_prev,dw,db

In [11]:
def lin_acti_backward(da,save,activation):
    lin_save,acti_save=save
    
    if activation=="relu":
        dz=relu_backward(da,acti_save)
        da_prev,dw,db=lin_backward(dz,lin_save)
        
    elif activation=="sigmoid":
        dz=sigmoid_backward(da,acti_save)
        da_prev,dw,db=lin_backward(dz,lin_save)
        
    return da_prev,dw,db

In [12]:
def l_backward_model(al,y,save_d):#save_d=deep layer's data
    
    
    grad={}
    
    L=len(save_d)
    m=al.shape[1]
    y=y.reshape(aL.shape)
    
    daL=-(y/aL-(1-y)/(1-aL))
    
    current_save=save_d[l-1]
    dal,dwl,dbl=lin_acti_backward(daL,current_save,"sigmoid")
    grad["da"+str(L)]=dal
    grad["dw"+str(L)]=dwl
    grad["db"+str(L)]=dbl
    
    for l in reversed(range(L-1)):
        current_save=save_d[l]
        dal,dwl,dbl=lin_acti_backward(dal,current_save,"relu")
        grad["da"+str(l+1)]=dal
        grad["dw"+str(l+1)]=dwl
        grad["db"+str(l+1)]=dbl
    
    return grad

    

In [ ]:
def update_params(params,grad,lr):
    L=len(params)//2
    for l in range(L):
        params["w"+str(l+1)]-=lr*grad["dw"+str(l+1)]
        params["b"+str(l+1)]-=lr*grad["db"+str(l+1)]
        
    return params